<a href="https://colab.research.google.com/github/karthikeyaniii/projects/blob/main/PRODIGY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [ ]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [ ]:
content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

In [ ]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

In [ ]:
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)

In [ ]:
import tensorflow_hub as hub
import os

# Specify the URL of the model and the path where you want to save it
model_url = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
save_path = '/content/sample_data/local_model'  # Path to save the model locally

# Check if the model is already downloaded to avoid re-downloading
if not os.path.exists(save_path):
    hub_model = hub.load(model_url)  # Load the model from TensorFlow Hub
    tf.saved_model.save(hub_model, save_path)  # Save the model locally
    print("Model downloaded and cached locally.")
else:
    print("Model already downloaded.")

In [ ]:
import shutil

# Zip the saved model directory
shutil.make_archive(save_path, 'zip', save_path)

print(f"Model saved and zipped at {save_path}.zip")

In [ ]:
import streamlit as st
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
import io
import urllib.request

# Function to load and preprocess images
def load_img(path_to_img):
    max_dim = 512
    if isinstance(path_to_img, bytes):
        img = tf.image.decode_image(path_to_img, channels=3)
    else:
        img = tf.io.read_file(path_to_img)
        img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

# Function to convert tensor to image
def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

# Load TensorFlow Hub model
@st.cache(allow_output_mutation=True)
def load_hub_model():
    model_url = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
    return hub.load(model_url)

def main():
    st.title('Neural Style Transfer')

    model = load_hub_model()

    st.sidebar.header('Upload Content and Style Images')
    uploaded_content_image = st.sidebar.file_uploader("Choose content image", type=["jpg", "jpeg", "png"])
    uploaded_style_image = st.sidebar.file_uploader("Choose style image", type=["jpg", "jpeg", "png"])

    content_url = st.sidebar.text_input('Or Enter URL for Content Image', '')
    style_url = st.sidebar.text_input('Or Enter URL for Style Image', '')

    if uploaded_content_image:
        content_image = load_img(uploaded_content_image.read())
    elif content_url:
        response = urllib.request.urlopen(content_url)
        content_image = load_img(response.read())
    else:
        st.sidebar.warning("Please upload an image or provide a URL for the content image.")
        return

    if uploaded_style_image:
        style_image = load_img(uploaded_style_image.read())
    elif style_url:
        response = urllib.request.urlopen(style_url)
        style_image = load_img(response.read())
    else:
        st.sidebar.warning("Please upload an image or provide a URL for the style image.")
        return

    st.image(tensor_to_image(content_image), caption='Content Image', use_column_width=True)
    st.image(tensor_to_image(style_image), caption='Style Image', use_column_width=True)

    if st.button('Apply Style Transfer'):
        stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]
        st.image(tensor_to_image(stylized_image), caption='Stylized Image', use_column_width=True)

if __name__ == "__main__":
    main()